# Project Set-up


In [1]:
try: 
    import pandas as pd
    import ta
    import numpy as np
    import requests
    import sys, os
    from dotenv import load_dotenv
    from pathlib import Path
    from datetime import datetime, timedelta
    
    print("Successfully Imported all the libraries")

except Exception as e: 
    print(f"Import Error: {e}")
    raise


Successfully Imported all the libraries


In [2]:
# PROJECT_ROOT = Path('/Users/shengyaotang/Desktop/Quant Project/ML_Portfolio_Management')
PROJECT_ROOT = Path('..')
DATA_DIR = PROJECT_ROOT/'data'

load_dotenv(PROJECT_ROOT/'.env')
print(".env file successfully loaded")


.env file successfully loaded


# Data Extraction: 
Extract S&P 500 healthcare data from wikipedia and financial modeling Prep Api

### Get Ticker data

In [10]:

alpha_vantage_api_key = os.getenv("ALPHA_VANTAGE_API_KEY")
etf_symbol = "DIA"

def get_qqq_holdings(api_key: str, symbol: str) -> pd.DataFrame:

    url = f'https://www.alphavantage.co/query?function=ETF_PROFILE&symbol={symbol}&apikey={api_key}'
    response = requests.get(url)
    
    if response.status_code == 200: 
        json_data = response.json()
        holdings_data = json_data["holdings"]
        df = pd.DataFrame(holdings_data)
        rows_to_drop = df[df['symbol'] == 'n/a'].index
        df = df.drop(rows_to_drop, axis = 0)

    else: 
        print("API request to Alpha Vantage failed.")
    df.to_csv(DATA_DIR/"raw"/"holdings.csv", index = False)
    # df.to_csv('../data/raw/holdings.csv', index = False)
    return df


In [11]:
df = get_qqq_holdings(alpha_vantage_api_key, etf_symbol)
holdings_list = df['symbol'].to_list()
print(holdings_list)

['GS', 'CAT', 'MSFT', 'AXP', 'V', 'HD', 'UNH', 'AMGN', 'SHW', 'JPM', 'MCD', 'IBM', 'TRV', 'AAPL', 'CRM', 'AMZN', 'BA', 'JNJ', 'HON', 'NVDA', 'MMM', 'CVX', 'PG', 'DIS', 'WMT', 'MRK', 'CSCO', 'KO', 'NKE', 'VZ']


### Extract historical price data from financial modeling prep

In [12]:
base_url = "https://financialmodelingprep.com"
data_type = "historical-price-eod"
ticker = "AAPL"
FMP_API_KEY = os.getenv("FMP_API_KEY")

url = f"{base_url}/stable/{data_type}/full?symbol={ticker}&apikey={FMP_API_KEY}"


In [13]:
def get_historical_price(url: str) -> pd.DataFrame: 
    try: 
        response = requests.get(url)
        print(response.status_code)
        data = response.json()
        df = pd.DataFrame(data, index = range(len(data)))
        print("Got historical price successfully")
    except Exception as e:
        # print(f"There is an error: {e}")
        df = pd.DataFrame()
        
    return df

In [14]:
def get_historical_prices(tickers: list, api_key: str) -> pd.DataFrame:

    df = pd.DataFrame()
    for ticker in tickers:
        url = f"{base_url}/stable/{data_type}/full?symbol={ticker}&apikey={api_key}"
        temp_df = get_historical_price(url)
        df = pd.concat([df,temp_df], axis = 0)

    # df.to_csv("/Users/shengyaotang/Desktop/Quant Project/ML_Portfolio_Management/data/raw/historical_price.csv", index = False)
    df.to_csv(DATA_DIR/"raw"/"historical_price.csv", index = False)
    # df.to_csv('../data/raw/historical_price.csv', index = False)
    return df

In [15]:
df = get_historical_prices(holdings_list, FMP_API_KEY)

200
Got historical price successfully
402
200
Got historical price successfully
402
200
Got historical price successfully
402
200
Got historical price successfully
402
402
200
Got historical price successfully
402
402
402
200
Got historical price successfully
402
200
Got historical price successfully
200
Got historical price successfully
200
Got historical price successfully
402
200
Got historical price successfully
402
200
Got historical price successfully
402
200
Got historical price successfully
200
Got historical price successfully
402
200
Got historical price successfully
200
Got historical price successfully
200
Got historical price successfully
200
Got historical price successfully


In [33]:
df.head(10)
df_holdings = pd.DataFrame(df["symbol"].unique(), columns = ["symbol"])
df_holdings.to_csv(DATA_DIR/"raw"/"holdings.csv", index = False)



# Data Transformation

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands

### Import the dataset

In [27]:
def load_data(filename): 
    df = pd.read_csv(filename)
    return df

In [28]:
# df = read_data(DATA_DIR)
df = load_data(DATA_DIR/"raw"/"historical_price.csv")
df.head(10)

,symbol,date,open,high,low,close,volume,change,changePercent,vwap
0,GS,2026-01-05,914.40,961.66,914.40,948.44,3222865,34.04,3.72266,941.5000
1,GS,2026-01-02,884.00,914.44,880.75,914.34,2817613,30.34,3.43000,898.3825
2,GS,2025-12-31,884.10,886.00,876.79,879.00,1221740,-5.10,-0.57686,881.4725
3,GS,2025-12-30,894.74,895.02,881.18,884.42,1833300,-10.32,-1.15000,888.8400
4,GS,2025-12-29,906.45,906.48,891.56,892.18,1578000,-14.27,-1.57000,899.1675
5,GS,2025-12-26,911.00,913.32,905.31,907.04,995328,-3.96,-0.43469,909.1675
6,GS,2025-12-24,901.16,911.88,898.70,910.78,771800,9.62,1.07000,905.6300
7,GS,2025-12-23,900.35,905.92,893.70,901.71,1210438,1.37,0.15105,900.4200
8,GS,2025-12-22,896.52,905.48,894.84,899.00,1442332,2.48,0.27663,898.9600
9,GS,2025-12-19,883.17,899.75,881.95,893.48,4808303,10.31,1.17000,889.5875


### Create Target Variable
1. Calculate weekly log returns

In [29]:
# Calculate weekly returns:
def calculate_weekly_returns(df: pd.DataFrame) -> pd.DataFrame:
    df['weekly_return'] = df.groupby('symbol')['close'].pct_change(periods = -5)
    df = df.dropna(subset = ["weekly_return"])
    
    df["weekly_log_return"] = np.log(1 + df["weekly_return"])
    
    return df

In [30]:
df = calculate_weekly_returns(df)
df.head(3)

/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_2267/387843821.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["weekly_log_return"] = np.log(1 + df["weekly_return"])


,symbol,date,open,high,low,close,volume,change,changePercent,vwap,weekly_return,weekly_log_return
0,GS,2026-01-05,914.4,961.66,914.40,948.44,3222865,34.04,3.72266,941.5000,0.045643,0.044632
1,GS,2026-01-02,884.0,914.44,880.75,914.34,2817613,30.34,3.43000,898.3825,0.003909,0.003901
2,GS,2025-12-31,884.1,886.00,876.79,879.00,1221740,-5.10,-0.57686,881.4725,-0.025185,-0.025508


2. 🎯 Create Target Variable

In [34]:
def create_target_variable(df: pd.DataFrame) -> pd.DataFrame:
    
    df = df.copy()
    
    df['date'] = pd.to_datetime(df['date'])
    df['year_week'] = df['date'].dt.strftime('%Y-%U')
    
    median_returns = df.groupby('year_week')['weekly_log_return'].median().reset_index()
    median_returns.columns = ['year_week', 'median_weekly_return']
    
    # Merge median returns back to original dataframe
    if 'median_weekly_return' not in df.columns:
        df = pd.merge(df, median_returns, on='year_week', how='left')
        
    df['target'] = (df['weekly_log_return'] > df['median_weekly_return']).astype(int)
    
    return df

In [35]:
df = create_target_variable(df)
df.head(3)

,symbol,date,open,high,low,close,volume,change,changePercent,vwap,weekly_return,weekly_log_return,year_week,median_weekly_return,target
0,GS,2026-01-05,914.4,961.66,914.40,948.44,3222865,34.04,3.72266,941.5000,0.045643,0.044632,2026-01,0.002320,1
1,GS,2026-01-02,884.0,914.44,880.75,914.34,2817613,30.34,3.43000,898.3825,0.003909,0.003901,2026-00,-0.002072,1
2,GS,2025-12-31,884.1,886.00,876.79,879.00,1221740,-5.10,-0.57686,881.4725,-0.025185,-0.025508,2025-52,0.004846,0


In [36]:
# 🗂 Create the panel dataset indexed by date and tickers

def create_panel_dataset(df: pd.DataFrame) -> pd.DataFrame:
    return df.sort_values(by = "date")

In [37]:
# This step could be moved to the pipeline
df = create_panel_dataset(df)
df.head(3)

,symbol,date,open,high,low,close,volume,change,changePercent,vwap,weekly_return,weekly_log_return,year_week,median_weekly_return,target
21232,VZ,2021-01-14,56.89,57.43,56.66,57.04,18341100,0.15,0.26367,57.0050,-0.025457,-0.025787,2021-02,-0.015096,0
1248,GS,2021-01-14,304.05,309.41,303.54,307.87,3192200,3.82,1.26000,306.2175,0.055615,0.054123,2021-02,-0.015096,1
18734,KO,2021-01-14,50.16,50.18,49.22,49.23,34303600,-0.93,-1.85000,49.6975,-0.014612,-0.014719,2021-02,-0.015096,1


In [38]:
# 🗑 Remove columns that will not be used for the project
def remove_columns(cols: list, df: pd.DataFrame) -> pd.DataFrame:
    try: 
        df = df.drop(columns = cols, axis = 1)
    except Exception as e:
        print(e)
    finally: 
        return df

In [39]:
cols = ["open", "high", "low", "change", 
        "changePercent", "weekly_return", 
       "year_week", "median_weekly_return"]
df = remove_columns(cols, df)
df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target
21232,VZ,2021-01-14,57.04,18341100,57.0050,-0.025787,0
1248,GS,2021-01-14,307.87,3192200,306.2175,0.054123,1
18734,KO,2021-01-14,49.23,34303600,49.6975,-0.014719,1


2. Calculate moving averages

In [40]:
#📈 Moving Averages

def calc_moving_avg(window: int, df: pd.DataFrame) -> pd.DataFrame:
    col_name = f'MA_{str(window)}'
    df[col_name] = df.groupby('symbol')['close'].transform(
        lambda x: x.rolling(window = window, min_periods = 1).mean()
    )
    
    return df


In [41]:
def calc_moving_avgs(windows: list, df: pd.DataFrame) -> pd.DataFrame: 
    for window in windows: 
        df = calc_moving_avg(window, df)
    return df

In [42]:
windows = [200, 100, 50]
df = calc_moving_avgs(windows, df)
df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50
21232,VZ,2021-01-14,57.04,18341100,57.0050,-0.025787,0,57.04,57.04,57.04
1248,GS,2021-01-14,307.87,3192200,306.2175,0.054123,1,307.87,307.87,307.87
18734,KO,2021-01-14,49.23,34303600,49.6975,-0.014719,1,49.23,49.23,49.23


### Risk and Volatility

In [43]:
def calc_volatility(col_name: str, window: int, df: pd.DataFrame) -> pd.DataFrame: 
    df[col_name] = df.groupby('symbol')['weekly_log_return'].transform(
        lambda x: x.rolling(window = window, min_periods = 1).std() * np.sqrt(window)
    )
    
    return df

In [44]:
def calc_volatilties(volatility_dict: dict, df: pd.DataFrame) -> pd.DataFrame: 
    for col_name, window in volatility_dict.items():
        df = calc_volatility(col_name, window, df)
    return df

In [45]:
volatility_dict = {
    'vol_1M': 4,    # 1 month ≈ 4 weeks
    'vol_6M': 26,   # 6 months ≈ 26 weeks
    'vol_12M': 52   # 12 months ≈ 52 weeks
}

df = calc_volatilties(volatility_dict, df)
#df = df.dropna(subset = ['vol_1M', 'vol_6M','vol_12M'])

df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,vol_1M,vol_6M,vol_12M
21232,VZ,2021-01-14,57.04,18341100,57.0050,-0.025787,0,57.04,57.04,57.04,NaN,NaN,NaN
1248,GS,2021-01-14,307.87,3192200,306.2175,0.054123,1,307.87,307.87,307.87,NaN,NaN,NaN
18734,KO,2021-01-14,49.23,34303600,49.6975,-0.014719,1,49.23,49.23,49.23,NaN,NaN,NaN


### Short Term Reversal Factors

In [46]:
def calculate_rsi(window, group): 
    rsi = RSIIndicator(close = group['close'], window = window).rsi()
    return rsi

In [47]:
def calculate_rsis(windows: list, df: pd.DataFrame) -> pd.DataFrame: 
    for window in windows: 
        col_name = f'RSI_{str(window)}'
        group = df.groupby('symbol', group_keys = False)
        df[col_name] = group.apply(lambda x: calculate_rsi(window, x))
        
    return df

In [48]:
windows = [3, 9, 14]
df = calculate_rsis(windows, df)
df.head(3)

/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_2267/1543531198.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[col_name] = group.apply(lambda x: calculate_rsi(window, x))
/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_2267/1543531198.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[col_name] = group.apply(lambda x: calculate_rsi(window, x))
/var/folders/gl/wt1cfyrs16358nbd8jfg3k70

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,vol_1M,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14
21232,VZ,2021-01-14,57.04,18341100,57.0050,-0.025787,0,57.04,57.04,57.04,NaN,NaN,NaN,NaN,NaN,NaN
1248,GS,2021-01-14,307.87,3192200,306.2175,0.054123,1,307.87,307.87,307.87,NaN,NaN,NaN,NaN,NaN,NaN
18734,KO,2021-01-14,49.23,34303600,49.6975,-0.014719,1,49.23,49.23,49.23,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
def calculate_bb(group, window = 20, window_dev = 2):
    return BollingerBands(close = group['close'], window = window, window_dev = window_dev)

In [50]:
def calculate_bbs(bands: list, df:pd.DataFrame) -> pd.DataFrame:
    group = df.groupby('symbol', group_keys = False)
    for band in bands:
        if band == "hband":
            df[band] = group.apply(lambda x: calculate_bb(x).bollinger_hband())
        elif band == "lband":
            df[band] = group.apply(lambda x: calculate_bb(x).bollinger_lband())
            
    return df

In [51]:
bands = ["hband", "lband"]
df = calculate_bbs(bands, df)
df.head(3)

/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_2267/3629412943.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[band] = group.apply(lambda x: calculate_bb(x).bollinger_hband())
/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_2267/3629412943.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[band] = group.apply(lambda x: calculate_bb(x).bollinger_lband())


,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,vol_1M,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14,hband,lband
21232,VZ,2021-01-14,57.04,18341100,57.0050,-0.025787,0,57.04,57.04,57.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1248,GS,2021-01-14,307.87,3192200,306.2175,0.054123,1,307.87,307.87,307.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18734,KO,2021-01-14,49.23,34303600,49.6975,-0.014719,1,49.23,49.23,49.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Momentum Factor: 

In [52]:
def calculate_momentum(window_months: int, df: pd.DataFrame, trading_days :int = 21) -> pd.DataFrame:
    window_days = window_months * trading_days
    col_name = f"momentum_{str(window_months)}M"
    df[col_name] = df.groupby('symbol')['close'].pct_change(periods = window_days)
    
    return df

In [53]:
def calculate_momentums(windows: list, df:pd.DataFrame) -> pd.DataFrame:
    for window in windows:
        df = calculate_momentum(window, df)
    return df

In [54]:
windows_for_momentum = [12, 6, 1]

df = calculate_momentums(windows_for_momentum, df)
df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,...,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14,hband,lband,momentum_12M,momentum_6M,momentum_1M
21232,VZ,2021-01-14,57.04,18341100,57.0050,-0.025787,0,57.04,57.04,57.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1248,GS,2021-01-14,307.87,3192200,306.2175,0.054123,1,307.87,307.87,307.87,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18734,KO,2021-01-14,49.23,34303600,49.6975,-0.014719,1,49.23,49.23,49.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Filter Datasets:

In [56]:
# 🗓Choose Each Wednesday and Thursday as the data transformation
def wed_thurs_selector(df, date_col='date', stock_col='symbol'):

    df['year_week'] = df[date_col].dt.strftime('%Y-%U')
    df['day_of_week'] = df[date_col].dt.dayofweek
    
    
    wed_thu = df[df['day_of_week'].isin([2, 3])].copy()
    
    
    filtered = wed_thu.groupby([stock_col, 'year_week']).first().reset_index()
    
    return filtered.drop(columns=['day_of_week', 'year_week'])
    

In [57]:
def filter_data(df: pd.DataFrame) -> pd.DataFrame: 
    df = df.copy()
    df = wed_thurs_selector(df)
    return df.dropna(ignore_index = True)

In [58]:
df = filter_data(df)
df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,...,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14,hband,lband,momentum_12M,momentum_6M,momentum_1M
0,AAPL,2022-01-12,175.53,74805200,175.9125,0.003481,1,146.12615,156.8118,166.2434,...,0.204168,0.247661,64.940264,52.297499,54.726541,182.735561,168.837439,0.335738,0.182179,-0.001195
1,AAPL,2022-01-19,166.23,94815000,168.3125,-0.051871,0,147.04630,157.6724,167.7942,...,0.164759,0.257659,11.730773,31.598820,39.999691,182.973032,167.483968,0.259032,0.143260,-0.028690
2,AAPL,2022-01-26,159.69,108275308,161.3500,-0.040138,0,147.80170,158.1554,169.0138,...,0.179705,0.298390,3.345620,22.148664,32.126858,184.839828,157.965172,0.124102,0.101462,-0.114457


In [59]:
def save_processed_data(df: pd.DataFrame, directory): 
    df.to_csv(directory/"processed"/"processed_historical_price.csv", index = False)

In [60]:
save_processed_data(df, DATA_DIR)

### Create Custom transformer
E.g. Remove/Fill in Nan values, log transformation, standardization

In [61]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

In [62]:
# Perform Log Transformation: 
class LogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        self.features = features

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for feature in self.features:
            X[feature] = np.log(X['close'] / X[feature])
        return X

### 💽Create training and testing variable

In [63]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import KFold

import joblib

In [64]:
# Create x and y variable

def create_variables(df: pd.DataFrame) -> list: 
    return [df.drop(columns = ["target"]), df["target"]]

X, y = create_variables(df)


In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [66]:
# symbols = df["symbol"].unique().tolist()

# X_train, X_test, y_train, y_test = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
# for symbol in symbols: 
#     symbol_df = df[df["symbol"] == symbol]
#     cut = int(len(symbol_df) * 0.8)
#     Xs, ys = create_variables(symbol_df)
#     Xs_train, Xs_test = Xs[:cut], Xs[cut:]
#     ys_train, ys_test = ys[:cut], ys[cut:]

#     X_train = pd.concat([X_train, Xs_train])
#     X_test = pd.concat([X_test, Xs_test])
#     y_train = pd.concat([y_train, ys_train])
#     y_test = pd.concat([y_test, ys_test])


def time_aware_split(df): 
    symbols = df["symbol"].unique().tolist()

    X_train, X_test, y_train, y_test = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    for symbol in symbols:
        symbol_df = df[df["symbol"] == symbol]
        cut = int(len(symbol_df) * 0.8)
        Xs, ys = create_variables(symbol_df)
        Xs_train, Xs_test = Xs[:cut], Xs[cut:]
        ys_train, ys_test = ys[:cut], ys[cut:]

        X_train = pd.concat([X_train, Xs_train])
        X_test = pd.concat([X_test, Xs_test])
        y_train = pd.concat([y_train, ys_train])
        y_test = pd.concat([y_test, ys_test])

    return X_train, X_test, y_train, y_test



In [67]:
X_train, X_test, y_train, y_test = time_aware_split(df)

In [68]:
# tscv = TimeSeriesSplit(n_splits=3)
# for i, (train_index, test_index) in enumerate(tscv.split(df)):
#     print(f"Fold {i + 1}:")
#     print(f"  Train: index={train_index}")
#     print(f"  Test:  index={test_index}")

### 🏭 Build a pipeline to transform features

In [69]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [70]:
#build pipeline
log_transform_features = ['MA_200', 'MA_100', 'MA_50', 'hband', 'lband']
# cols_to_drop = ['Ticker', 'date', 'weekly_log_return']

cols_to_drop = ['symbol', 'date', 'weekly_log_return']

# Example: Drop columns 'col1' and 'col2'
col_dropper = ColumnTransformer(
    transformers=[
        ('drop_cols', 'drop', cols_to_drop)
    ],
    remainder='passthrough'  # keeps all other columns
)

logistic_regression_pipeline = Pipeline(steps = [ 
    ('log_transformers', LogTransformer(log_transform_features)),
    ('col_dropper', col_dropper), 
    ('scaler', StandardScaler()), 
    ('classifier', LogisticRegression())
])

In [71]:
logistic_regression_model = logistic_regression_pipeline.fit(X_train, y_train)

/Users/shengyaotang/opt/anaconda3/envs/bootcamp_env/lib/python3.10/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [72]:
filename = "logistic_regression_model.sav"
joblib.dump(logistic_regression_model, open(filename, 'wb'))

In [73]:
logistic_regression_model = joblib.load(open(filename, 'rb'))
y_pred = logistic_regression_model.predict(X_test)
print(y_pred)

[0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1.
 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1.
 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1.
 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1.
 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0.
 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0.
 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1.
 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1.
 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0.
 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0.
 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0.
 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1.
 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1.

In [74]:
print(f"Model accuracy: {logistic_regression_model.score(X_test, y_test)}")

Model accuracy: 0.7689075630252101


# Create Weekly Stock Portfolio

In [75]:
def get_last_week_data(filename):
    df = load_data(filename)
    df['date'] = pd.to_datetime(df['date'])
    df['year_week'] = df['date'].dt.strftime('%Y-%U')
    last_year_week = (datetime.now() - timedelta(weeks = 1)).strftime('%Y-%U')

    df_last_week = df[df['year_week'] == last_year_week]
    return df_last_week.drop(columns=['year_week'])

In [76]:
filename = DATA_DIR/'processed'/'processed_historical_price.csv'
df_last_week = get_last_week_data(filename).reset_index(drop = True)
X_last_week = df_last_week.drop(columns = ['target'])
y_last_week = df_last_week['target']


df_last_week

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,...,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14,hband,lband,momentum_12M,momentum_6M,momentum_1M
0,AAPL,2025-12-31,271.86,27293639,272.5875,-0.001837,0,231.75030,257.1952,272.6622,...,0.115245,0.174165,25.690566,40.340241,45.642631,282.346116,268.598884,0.063657,0.279703,-0.039703
1,AMZN,2025-12-31,230.82,24383749,231.7100,-0.005702,0,216.86010,228.4211,231.4368,...,0.154135,0.342598,37.994599,53.622142,51.997702,235.750456,221.794544,0.031598,0.049563,-0.013084
2,BA,2025-12-31,217.12,5563028,217.8775,0.001244,0,205.26490,212.7044,202.6852,...,0.234012,0.387150,53.405985,69.222007,65.447893,222.782120,195.096880,0.201417,0.024006,0.164432
3,CSCO,2025-12-31,77.03,13570500,77.2675,-0.012770,0,67.65820,71.9373,75.6440,...,0.114423,0.226802,16.282592,43.504052,49.306238,79.873365,76.214635,0.292233,0.123050,0.013019
4,CVX,2025-12-31,152.41,4953110,152.2550,0.012545,1,150.26440,154.0208,152.1410,...,0.086947,0.129950,86.123628,61.301648,55.441911,153.156751,146.965249,0.058403,0.029936,-0.000852
5,DIS,2025-12-31,113.77,6608000,114.2450,0.004846,0,110.12850,112.2431,109.7554,...,0.108150,0.229413,47.359577,61.800053,59.868772,116.788002,104.740998,0.019901,-0.074890,0.065561
6,GS,2025-12-31,879.00,1221740,881.4725,-0.025508,0,705.98485,796.3761,827.2266,...,0.158054,0.244096,17.658747,49.156891,55.180402,923.589268,840.954732,0.525565,0.227842,0.084034
7,JNJ,2025-12-31,206.95,4084013,206.9325,0.005670,1,173.91725,190.3784,199.8958,...,0.131325,0.188557,38.676875,50.848176,54.834095,213.679316,200.106684,0.426749,0.330355,0.007841
8,JPM,2025-12-31,322.22,5048531,323.3700,-0.011448,0,285.56035,306.8804,310.9000,...,0.104014,0.195003,32.624740,55.405654,56.370508,331.192036,304.942964,0.336070,0.103493,0.043053
9,KO,2025-12-31,69.91,8127801,70.0050,0.000572,0,69.90320,69.1536,70.4762,...,0.094286,0.166766,35.860622,43.641960,46.075009,70.977587,69.429413,0.119456,-0.014102,-0.028353


In [77]:
y_pred = logistic_regression_model.predict(X_last_week)
y_prob = logistic_regression_model.predict_proba(X_last_week)

df_pred = pd.DataFrame(y_pred, columns = ['prediction'])
df_prob = pd.DataFrame(y_prob, columns = ['underperform', 'outperform'])

df_pred

,prediction
0,0.0
1,0.0
2,1.0
3,0.0
4,1.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0


In [78]:
def create_weekly_stock_portfolio(*args: pd.DataFrame) -> pd.DataFrame:
    df_stock_portfolio = pd.concat(args, axis = 1)
    df_stock_portfolio = df_stock_portfolio.sort_values(by = ['outperform'], ascending = False)
    df_stock_portfolio = df_stock_portfolio[df_stock_portfolio['prediction'] == 1]
    df_stock_portfolio['weight'] = 1 / len(df_stock_portfolio)
    df_stock_portfolio.to_csv(DATA_DIR/'processed'/'stock_portfolio.csv', index = False)
    return df_stock_portfolio.reset_index(drop = True)

In [79]:
df_stock_portfolio = create_weekly_stock_portfolio(df_last_week['symbol'], df_prob, df_pred)
df_stock_portfolio

,symbol,underperform,outperform,prediction,weight
0,NKE,0.060980,0.939020,1.0,0.2
1,CVX,0.129757,0.870243,1.0,0.2
2,VZ,0.179223,0.820777,1.0,0.2
3,NVDA,0.444056,0.555944,1.0,0.2
4,BA,0.446630,0.553370,1.0,0.2


In [1]:
def update_stock_portfolio(df_stock_portfolio: pd.DataFrame, df_historical_price: pd.DataFrame) -> pd.DataFrame:
    df_historical_price = df_historical_price.copy()
    
    df_historical_price['date'] = pd.to_datetime(df_historical_price['date'])
    df_historical_price['week_of_year'] = df_historical_price['date'].dt.strftime('%Y-%U')
    df_weekly_price = df_historical_price[df_historical_price['week_of_year'] == datetime.now().strftime('%Y-%U') ]

    if 'close' not in df_stock_portfolio.columns: 
        df_weekly_portfolio = pd.merge(df_stock_portfolio, df_weekly_price, on = 'symbol', how = 'left')
    df_weekly_portfolio['week_of_day'] = df_weekly_portfolio['date'].dt.weekday
    df_weekly_portfolio.to_csv(DATA_DIR/'processed'/'weekly_portfolio.csv')

    return df_weekly_portfolio.dropna()

NameError: name 'pd' is not defined

In [128]:
df_historical_price = load_data(DATA_DIR/'raw'/'historical_price.csv')
df_weekly_portfolio = update_stock_portfolio(df_stock_portfolio, df_historical_price)
df_weekly_portfolio

,symbol,underperform,outperform,prediction,weight,date,open,high,low,close,volume,change,changePercent,vwap,week_of_year,week_of_day
0,NKE,0.060980,0.939020,1.0,0.2,2026-01-05,63.740,65.3700,63.3900,64.53,23482539,0.795,1.240000,64.2575,2026-01,0
1,CVX,0.129757,0.870243,1.0,0.2,2026-01-05,165.750,165.7500,159.3098,163.87,32553766,-1.880,-1.134240,162.9800,2026-01,0
2,VZ,0.179223,0.820777,1.0,0.2,2026-01-05,40.430,40.5150,39.8300,40.23,43057941,-0.200,-0.494682,40.1900,2026-01,0
3,NVDA,0.444056,0.555944,1.0,0.2,2026-01-05,191.715,193.6300,186.1500,188.12,178569651,-3.590,-1.875180,189.3000,2026-01,0
4,BA,0.446630,0.553370,1.0,0.2,2026-01-05,228.790,230.9599,227.5000,228.13,7173157,-0.660,-0.288474,228.8600,2026-01,0


In [104]:
def calculate_portfolio_metrics(portfolio: pd.DataFrame) -> pd.DataFrame:
    s = portfolio.groupby("date")['close'].sum()
    df = pd.DataFrame(s, columns=['close']).reset_index()
    df['daily_return'] = df['close'].pct_change()
    df.loc[0, 'daily_return'] = 0
    df['cumulative_return'] = (df['close'] - df['close'][0]) / df['close'][0] 

    return df

# s_portfolio_metric = df_weekly_portfolio.groupby("date")['close'].sum()
# df_portfolio_metric = pd.DataFrame(s_portfolio_metric, columns=['close']).reset_index()

In [132]:
# class Portfolio_Tracker:
#     def __init__(self, portfolio, start_date='2026-01-01'):
#         self.portfolio = portfolio
#         self.start_date = start_date
# total_value = 1000000
# if len(df_weekly_portfolio) != 0:
#     df_weekly_portfolio['asset_allocated'] = total_value * df_weekly_portfolio['weight'] 
#     df_weekly_portfolio.loc[df_weekly_portfolio['week_of_day'] == 0, 'shares_per_stock'] = df_weekly_portfolio['asset_allocated'] / df_weekly_portfolio['close']
    

# df_weekly_portfolio




,symbol,underperform,outperform,prediction,weight,date,open,high,low,close,volume,change,changePercent,vwap,week_of_year,week_of_day,asset_allocated,shares_per_stock
0,NKE,0.060980,0.939020,1.0,0.2,2026-01-05,63.740,65.3700,63.3900,64.53,23482539,0.795,1.240000,64.2575,2026-01,0,200000.0,3099.333643
1,CVX,0.129757,0.870243,1.0,0.2,2026-01-05,165.750,165.7500,159.3098,163.87,32553766,-1.880,-1.134240,162.9800,2026-01,0,200000.0,1220.479649
2,VZ,0.179223,0.820777,1.0,0.2,2026-01-05,40.430,40.5150,39.8300,40.23,43057941,-0.200,-0.494682,40.1900,2026-01,0,200000.0,4971.414367
3,NVDA,0.444056,0.555944,1.0,0.2,2026-01-05,191.715,193.6300,186.1500,188.12,178569651,-3.590,-1.875180,189.3000,2026-01,0,200000.0,1063.151180
4,BA,0.446630,0.553370,1.0,0.2,2026-01-05,228.790,230.9599,227.5000,228.13,7173157,-0.660,-0.288474,228.8600,2026-01,0,200000.0,876.693114


In [83]:
df_portfolio_value = calculate_portfolio_metrics(df_weekly_portfolio)
df_portfolio_value

,date,close,daily_return,cumulative_return
0,2026-01-05,684.88,0.0,0.0


In [ ]:
#create a new function that tracks the historical performance. 
#create the new function by merging the calculate weekly portfolio metrics function
